In [1]:
import kagglehub

path = kagglehub.dataset_download("tschandl/isic2018-challenge-task1-data-segmentation")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/datasets/tschandl/isic2018-challenge-task1-data-segmentation


In [2]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

import albumentations as A
from albumentations.pytorch import ToTensorV2

# KaggleHub dataset path
BASE_DIR = "/kaggle/input/datasets/tschandl/isic2018-challenge-task1-data-segmentation"

print("Available folders:")
print(os.listdir(BASE_DIR))

TRAIN_IMG_DIR = os.path.join(BASE_DIR, "ISIC2018_Task1-2_Training_Input")
TRAIN_MASK_DIR = os.path.join(BASE_DIR, "ISIC2018_Task1_Training_GroundTruth")

print("Image path exists:", os.path.exists(TRAIN_IMG_DIR))
print("Mask path exists:", os.path.exists(TRAIN_MASK_DIR))

IMG_SIZE = 224
BATCH_SIZE = 4
LR = 1e-4
EPOCHS = 15
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("Using device:", DEVICE)


Available folders:
['ISIC2018_Task1-2_Training_Input', 'ISIC2018_Task1_Training_GroundTruth', 'ISIC2018_Task1-2_Test_Input', 'ISIC2018_Task1-2_Validation_Input']
Image path exists: True
Mask path exists: True
Using device: cuda


In [3]:
class ISICDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_paths = sorted(glob(os.path.join(image_dir, "*.jpg")))
        self.mask_dir = mask_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        filename = os.path.basename(img_path).replace(".jpg", "_segmentation.png")
        mask_path = os.path.join(self.mask_dir, filename)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented["image"]
            mask = augmented["mask"]

        mask = mask.unsqueeze(0).float() / 255.0

        return image, mask


In [4]:
transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(),
    ToTensorV2(),
])


In [5]:
def dice_loss(pred, target, smooth=1):
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum()
    dice = (2 * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    return 1 - dice


class DiceBCELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, pred, target):
        return self.bce(pred, target) + dice_loss(pred, target)


In [6]:
dataset = ISICDataset(TRAIN_IMG_DIR, TRAIN_MASK_DIR, transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

print("Train samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))


Train samples: 2075
Validation samples: 519


In [7]:
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)


class UNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.enc1 = DoubleConv(3, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)

        self.pool = nn.MaxPool2d(2)
        self.bottleneck = DoubleConv(512, 1024)

        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = DoubleConv(1024, 512)

        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = DoubleConv(512, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = DoubleConv(256, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = DoubleConv(128, 64)

        self.final = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        e4 = self.enc4(self.pool(e3))

        b = self.bottleneck(self.pool(e4))

        d4 = self.up4(b)
        d4 = torch.cat([d4, e4], dim=1)
        d4 = self.dec4(d4)

        d3 = self.up3(d4)
        d3 = torch.cat([d3, e3], dim=1)
        d3 = self.dec3(d3)

        d2 = self.up2(d3)
        d2 = torch.cat([d2, e2], dim=1)
        d2 = self.dec2(d2)

        d1 = self.up1(d2)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)

        return self.final(d1)


In [8]:
def dice_loss(pred, target, smooth=1):
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum()
    dice = (2 * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    return 1 - dice


class DiceBCELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, pred, target):
        return self.bce(pred, target) + dice_loss(pred, target)


In [9]:
dataset = ISICDataset(TRAIN_IMG_DIR, TRAIN_MASK_DIR, transform)

print("Total images found:", len(dataset))

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0   # important for stability
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0
)

print("Train samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))


Total images found: 2594
Train samples: 2075
Validation samples: 519


In [10]:
model = UNet().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = DiceBCELoss()

for epoch in range(EPOCHS):
    torch.cuda.empty_cache()

    # -------- TRAINING --------
    model.train()
    train_loss = 0

    for images, masks in tqdm(train_loader):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # -------- VALIDATION --------
    model.eval()

    dice_score = 0
    iou_score = 0

    TP = 0
    FP = 0
    FN = 0
    TN = 0

    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)

            outputs = torch.sigmoid(model(images))
            outputs = (outputs > 0.5).float()

            # Flatten tensors
            outputs_flat = outputs.view(-1)
            masks_flat = masks.view(-1)

            # Confusion components
            tp = (outputs_flat * masks_flat).sum()
            fp = (outputs_flat * (1 - masks_flat)).sum()
            fn = ((1 - outputs_flat) * masks_flat).sum()
            tn = ((1 - outputs_flat) * (1 - masks_flat)).sum()

            TP += tp.item()
            FP += fp.item()
            FN += fn.item()
            TN += tn.item()

            # Dice & IoU
            intersection = tp
            union = tp + fp + fn

            dice = (2 * intersection) / (2 * intersection + fp + fn + 1e-8)
            iou = intersection / (union + 1e-8)

            dice_score += dice.item()
            iou_score += iou.item()

    dice_score /= len(val_loader)
    iou_score /= len(val_loader)

    # Derived metrics
    pixel_accuracy = (TP + TN) / (TP + TN + FP + FN + 1e-8)
    precision = TP / (TP + FP + 1e-8)
    recall = TP / (TP + FN + 1e-8)          # Sensitivity
    specificity = TN / (TN + FP + 1e-8)

    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Dice: {dice_score:.4f}")
    print(f"Val IoU: {iou_score:.4f}")
    print(f"Pixel Acc: {pixel_accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")


100%|██████████| 519/519 [08:46<00:00,  1.01s/it]



Epoch 1/15
Train Loss: 0.8999
Val Dice: 0.7390
Val IoU: 0.6082
Pixel Acc: 0.8794
Precision: 0.6782
Recall (Sensitivity): 0.8161
Specificity: 0.8963


100%|██████████| 519/519 [07:15<00:00,  1.19it/s]



Epoch 2/15
Train Loss: 0.6376
Val Dice: 0.8023
Val IoU: 0.6840
Pixel Acc: 0.9190
Precision: 0.7897
Recall (Sensitivity): 0.8401
Specificity: 0.9401


100%|██████████| 519/519 [07:14<00:00,  1.20it/s]


...


Epoch 13/15
Train Loss: 0.3255
Val Dice: 0.8635
Val IoU: 0.7698
Pixel Acc: 0.9478
Precision: 0.9140
Recall (Sensitivity): 0.8313
Specificity: 0.9790


100%|██████████| 519/519 [07:10<00:00,  1.21it/s]



Epoch 14/15
Train Loss: 0.3105
Val Dice: 0.8516
Val IoU: 0.7537
Pixel Acc: 0.9444
Precision: 0.9530
Recall (Sensitivity): 0.7748
Specificity: 0.9898


100%|██████████| 519/519 [07:14<00:00,  1.20it/s]



Epoch 15/15
Train Loss: 0.3069
Val Dice: 0.8684
Val IoU: 0.7753
Pixel Acc: 0.9488
Precision: 0.8886
Recall (Sensitivity): 0.8659
Specificity: 0.9709


In [11]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, "unet_isic_checkpoint.pth")

print("Checkpoint saved.")


Checkpoint saved.


In [12]:
from IPython.display import FileLink
FileLink('/kaggle/working/unet_isic_checkpoint.pth')


/kaggle/working/unet_isic_checkpoint.pth

In [16]:
!zip model_checkpoint.zip /kaggle/working/unet_isic_checkpoint.pth


  adding: kaggle/working/unet_isic_checkpoint.pth (deflated 8%)


In [17]:
from IPython.display import FileLink
display(FileLink('model_checkpoint.zip'))


/kaggle/working/model_checkpoint.zip

In [4]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tkinter import Tk
from tkinter.filedialog import askopenfilename

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMG_SIZE = 224  # Must match training size


# ---------- U-Net Architecture (Must Match Training) ----------

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)


class UNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.enc1 = DoubleConv(3, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)

        self.pool = nn.MaxPool2d(2)
        self.bottleneck = DoubleConv(512, 1024)

        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = DoubleConv(1024, 512)

        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = DoubleConv(512, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = DoubleConv(256, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = DoubleConv(128, 64)

        self.final = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        e4 = self.enc4(self.pool(e3))

        b = self.bottleneck(self.pool(e4))

        d4 = self.up4(b)
        d4 = torch.cat([d4, e4], dim=1)
        d4 = self.dec4(d4)

        d3 = self.up3(d4)
        d3 = torch.cat([d3, e3], dim=1)
        d3 = self.dec3(d3)

        d2 = self.up2(d3)
        d2 = torch.cat([d2, e2], dim=1)
        d2 = self.dec2(d2)

        d1 = self.up1(d2)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)

        return self.final(d1)


# ---------- Load Model ----------

model = UNet().to(DEVICE)

checkpoint_path = r"C:\Users\dines\Downloads\model_checkpoint (1)\kaggle\working\unet_isic_checkpoint.pth"

checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

print("Model loaded successfully.")


# ---------- Select Image From PC ----------

Tk().withdraw()
image_path = askopenfilename(title="Select Skin Image")

if image_path == "":
    print("No image selected.")
    exit()

print("Selected image:", image_path)


# ---------- Preprocess Image ----------

image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
original_image = image_rgb.copy()

image_resized = cv2.resize(image_rgb, (IMG_SIZE, IMG_SIZE))
image_norm = image_resized / 255.0

image_tensor = torch.tensor(image_norm, dtype=torch.float32)
image_tensor = image_tensor.permute(2, 0, 1).unsqueeze(0).to(DEVICE)


# ---------- Predict ----------

with torch.no_grad():
    output = torch.sigmoid(model(image_tensor))
    mask = (output > 0.5).float()

mask = mask.squeeze().cpu().numpy()


# ---------- Display Results ----------

plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.title("Original")
plt.imshow(original_image)
plt.axis("off")

plt.subplot(1,3,2)
plt.title("Predicted Mask")
plt.imshow(mask, cmap="gray")
plt.axis("off")

plt.subplot(1,3,3)
plt.title("Overlay")
plt.imshow(original_image)
plt.imshow(mask, cmap="jet", alpha=0.4)
plt.axis("off")

plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\dines\\Downloads\\model_checkpoint (1)\\kaggle\\working\\unet_isic_checkpoint.pth'